In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import classification_report 
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline 
import matplotlib as plt 

path = 'C:/Users/e745092/Desktop/Visual Studio/OR_Cases_2020.csv'

data = pd.read_csv(path)
data.head()

,Lead Surgeon,Date,MRN,Diagnosis Codes,Case Procedures,Service,Case Indx,Primary Categ,Sub Categ
0,142333,1/6/2020,2348312,Gastric adenocarcinoma (HCC) [C16.9],Laparoscopic Subtotal Gastrectomy With Roux-En...,General,Y,Esophagus,G
1,131184,1/6/2020,8705179,Chronic fibrosing pancreatitis (HCC) [K86.1],"Total Pancreatectomy, Auto Islet Cell Transpla...",Transplant,Y,Pancreas,P
2,152600,1/8/2020,15415485,Malignant neoplasm of abdominal esophagus (HCC...,Minimally Invasive Esophagectomy Ivor Lewis [4...,Thoracic,Y,Esophagus,E
3,152600,1/9/2020,14936925,"History of esophagectomy [Z98.890, Z90.49]",Esophagogastroduodenoscopy W/ Dilation [78],Thoracic,Y,Esophagus,E-HxE
4,122994,1/10/2020,8796688,Malignant neoplasm of lesser curvature of stom...,"Egd [688], Laparoscopic Distal Gastrectomy Wi...",General,Y,Gastric,G


In [2]:
data.shape

(335, 9)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Lead Surgeon     335 non-null    int64 
 1   Date             335 non-null    object
 2   MRN              334 non-null    object
 3   Diagnosis Codes  335 non-null    object
 4   Case Procedures  335 non-null    object
 5   Service          335 non-null    object
 6   Case Indx        334 non-null    object
 7   Primary Categ    335 non-null    object
 8   Sub Categ        335 non-null    object
dtypes: int64(1), object(8)
memory usage: 13.1+ KB


In [4]:
data.isnull().sum()

Lead Surgeon       0
Date               0
MRN                1
Diagnosis Codes    0
Case Procedures    0
Service            0
Case Indx          1
Primary Categ      0
Sub Categ          0
dtype: int64

In [5]:
data = data.dropna()

In [6]:
import re

data['DX1']=data['Diagnosis Codes'].apply(lambda x: re.findall('\[(.*?)\]',x))
# print(data['DX'])
data['Procedure Codes1']=data['Case Procedures'].apply(lambda x: re.findall('\[(.*?)\]',x))
#print(data['Procedure Codes'])

In [7]:
print(type(data.loc[0, 'Procedure Codes1']))

<class 'list'>


In [8]:
data['DX_Codes'] = data['DX1'].str.get(0)
# data['Procedure_Codes'] = data['Procedure Codes1'].str.get(0)
data['Procedure_Codes'] =  data['Procedure Codes1'].str.join(', ')
data.head()

,Lead Surgeon,Date,MRN,Diagnosis Codes,Case Procedures,Service,Case Indx,Primary Categ,Sub Categ,DX1,Procedure Codes1,DX_Codes,Procedure_Codes
0,142333,1/6/2020,2348312,Gastric adenocarcinoma (HCC) [C16.9],Laparoscopic Subtotal Gastrectomy With Roux-En...,General,Y,Esophagus,G,[C16.9],"[1075612297, 1075612211]",C16.9,"1075612297, 1075612211"
1,131184,1/6/2020,8705179,Chronic fibrosing pancreatitis (HCC) [K86.1],"Total Pancreatectomy, Auto Islet Cell Transpla...",Transplant,Y,Pancreas,P,[K86.1],"[1075612630, 1075611937]",K86.1,"1075612630, 1075611937"
2,152600,1/8/2020,15415485,Malignant neoplasm of abdominal esophagus (HCC...,Minimally Invasive Esophagectomy Ivor Lewis [4...,Thoracic,Y,Esophagus,E,[C15.5],"[43287, 1075617378, 688]",C15.5,"43287, 1075617378, 688"
3,152600,1/9/2020,14936925,"History of esophagectomy [Z98.890, Z90.49]",Esophagogastroduodenoscopy W/ Dilation [78],Thoracic,Y,Esophagus,E-HxE,"[Z98.890, Z90.49]",[78],"Z98.890, Z90.49",78
4,122994,1/10/2020,8796688,Malignant neoplasm of lesser curvature of stom...,"Egd [688], Laparoscopic Distal Gastrectomy Wi...",General,Y,Gastric,G,[C16.5],"[688, 1070000770]",C16.5,"688, 1070000770"


In [9]:
data.isnull().any()

Lead Surgeon        False
Date                False
MRN                 False
Diagnosis Codes     False
Case Procedures     False
Service             False
Case Indx           False
Primary Categ       False
Sub Categ           False
DX1                 False
Procedure Codes1    False
DX_Codes            False
Procedure_Codes     False
dtype: bool

In [10]:
X = data.drop(['Lead Surgeon', 'Date', 'MRN', 'Diagnosis Codes', 'Case Procedures', 'DX1', 'Procedure Codes1', 'Primary Categ'], axis=1)
y = data['Primary Categ']
print(X.shape)
print(y.shape)

(333, 5)
(333,)


In [11]:
print(X.head())
print(y.unique())

      Service Case Indx Sub Categ         DX_Codes         Procedure_Codes
0     General         Y         G            C16.9  1075612297, 1075612211
1  Transplant         Y         P            K86.1  1075612630, 1075611937
2    Thoracic         Y         E            C15.5  43287, 1075617378, 688
3    Thoracic         Y     E-HxE  Z98.890, Z90.49                      78
4     General         Y         G            C16.5         688, 1070000770
['Esophagus' 'Pancreas' 'Gastric' 'Other' 'Lung' 'O']


# OVR SVC model 

In [12]:
from sklearn.preprocessing import label_binarize 

# y6 = label_binarize(y, classes=['Esophagus', 'Pancreas', 'Gastric', 'Other', 'Lung', 'O'])

hot_encoder = OneHotEncoder()
X_encoded = hot_encoder.fit_transform(X)

print(X_encoded.shape)
print(y.shape)

(333, 247)
(333,)


In [13]:
from sklearn.model_selection import train_test_split

# I shouldn't fit_transform test data. I should split before encoding. 

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42) # train with y instead of y6 which is encoded to seperate each class. 

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(266, 247)
(67, 247)
(266,)
(67,)


In [14]:
ovr_clf = OneVsRestClassifier(SVC(degree=2, gamma="scale", random_state=42))

ovr_clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(degree=2, random_state=42))

In [15]:
print(y_test.unique())
print(y_train.unique())

['Gastric' 'Lung' 'Pancreas' 'Esophagus']
['Gastric' 'Pancreas' 'Esophagus' 'Lung' 'Other' 'O']


In [17]:
# Use test data. 

print(classification_report(y_test, ovr_clf.predict(X_test), target_names=['Esophagus', 'Pancreas', 'Gastric', 'Lung']))

              precision    recall  f1-score   support

   Esophagus       0.89      1.00      0.94        25
    Pancreas       1.00      0.75      0.86        12
     Gastric       1.00      1.00      1.00        25
        Lung       1.00      1.00      1.00         5

    accuracy                           0.96        67
   macro avg       0.97      0.94      0.95        67
weighted avg       0.96      0.96      0.95        67



In [18]:
test_path = 'C:/Users/e745092/Desktop/Visual Studio/test.csv'

test_data = pd.read_csv(test_path)
test_data.head()

,Diagnosis Codes,Case Procedures,Service,Case Indx,Sub Categ
0,Gastric adenocarcinoma (HCC) [C16.9],Laparoscopic Subtotal Gastrectomy With Roux-En...,General,Y,G
1,Chronic fibrosing pancreatitis (HCC) [K86.1],"Total Pancreatectomy, Auto Islet Cell Transpla...",Transplant,Y,P
2,Malignant neoplasm of abdominal esophagus (HCC...,Minimally Invasive Esophagectomy Ivor Lewis [4...,Thoracic,Y,E
3,"History of esophagectomy [Z98.890, Z90.49]",Esophagogastroduodenoscopy W/ Dilation [78],Thoracic,Y,E-HxE
4,Malignant neoplasm of lesser curvature of stom...,"Egd [688], Laparoscopic Distal Gastrectomy Wi...",General,Y,G


In [19]:
test_data['DX1']=test_data['Diagnosis Codes'].apply(lambda x: re.findall('\[(.*?)\]',x))
# print(data['DX'])
test_data['Procedure Codes1']=test_data['Case Procedures'].apply(lambda x: re.findall('\[(.*?)\]',x))
#print(data['Procedure Codes'])

test_data['DX_Codes'] = test_data['DX1'].str.get(0)
test_data['Procedure_Codes'] =  test_data['Procedure Codes1'].str.join(', ')
test_data.head()

,Diagnosis Codes,Case Procedures,Service,Case Indx,Sub Categ,DX1,Procedure Codes1,DX_Codes,Procedure_Codes
0,Gastric adenocarcinoma (HCC) [C16.9],Laparoscopic Subtotal Gastrectomy With Roux-En...,General,Y,G,[C16.9],"[1075612297, 1075612211]",C16.9,"1075612297, 1075612211"
1,Chronic fibrosing pancreatitis (HCC) [K86.1],"Total Pancreatectomy, Auto Islet Cell Transpla...",Transplant,Y,P,[K86.1],"[1075612630, 1075611937]",K86.1,"1075612630, 1075611937"
2,Malignant neoplasm of abdominal esophagus (HCC...,Minimally Invasive Esophagectomy Ivor Lewis [4...,Thoracic,Y,E,[C15.5],"[43287, 1075617378, 688]",C15.5,"43287, 1075617378, 688"
3,"History of esophagectomy [Z98.890, Z90.49]",Esophagogastroduodenoscopy W/ Dilation [78],Thoracic,Y,E-HxE,"[Z98.890, Z90.49]",[78],"Z98.890, Z90.49",78
4,Malignant neoplasm of lesser curvature of stom...,"Egd [688], Laparoscopic Distal Gastrectomy Wi...",General,Y,G,[C16.5],"[688, 1070000770]",C16.5,"688, 1070000770"


In [20]:
test_data = test_data.drop(['DX1', 'Procedure Codes1','Diagnosis Codes', 'Case Procedures'], axis=1)
test_data.head(10)

,Service,Case Indx,Sub Categ,DX_Codes,Procedure_Codes
0,General,Y,G,C16.9,"1075612297, 1075612211"
1,Transplant,Y,P,K86.1,"1075612630, 1075611937"
2,Thoracic,Y,E,C15.5,"43287, 1075617378, 688"
3,Thoracic,Y,E-HxE,"Z98.890, Z90.49",78
4,General,Y,G,C16.5,"688, 1070000770"
5,Thoracic,Y,O,R13.10,78
6,Thoracic,Y,O,C15.5,688
7,Thoracic,N,LM,K44.9,
8,Thoracic,Y,O,C15.5,1075617077


In [21]:
test_data.isnull().any()

Service            False
Case Indx          False
Sub Categ          False
DX_Codes           False
Procedure_Codes    False
dtype: bool

In [24]:
ignore_encoder = OneHotEncoder(handle_unknown='ignore')
ignore_encoder.fit_transform(X)


test_data_encoded = ignore_encoder.transform(test_data)

test_data_encoded.shape

(9, 247)

In [26]:
ovr_clf_prediction = ovr_clf.predict(test_data_encoded)
pd.Series(ovr_clf_prediction)

0      Gastric
1     Pancreas
2    Esophagus
3    Esophagus
4      Gastric
5    Esophagus
6    Esophagus
7    Esophagus
8    Esophagus
dtype: object

In [33]:
ovr_clf.predict(test_data_encoded[[0]])

array(['Gastric'], dtype='<U9')

In [42]:
path_2 = 'C:/Users/e745092/Desktop/Visual Studio/test_2.csv'
test_2 = pd.read_csv(path_2)
test_2.head()


,Diagnosis Codes,Case Procedures,Service,Case Indx,Sub Categ
0,Pancreatic mass [K86.89],Laparoscopic Distal Pancreatectomy And Splenec...,General,Y,P-Distal
1,Ductal carcinoma in situ of right breast [D05.11],Right Lumpectomy [1075617330],General,Y,B-Partial M
2,Other chronic pancreatitis (HCC) [K86.1],"Total Pancreatectomy, Auto Islet Cell Transpla...",Transplant,Y,P-Total
3,Malignant neoplasm of lower-outer quadrant of ...,"Bilateral Total Mastectomy [1075617332], Right...",General,Y,B-M
4,Malignant neoplasm of lower third of esophagus...,"Laparoscopic Cholecystectomy, Pyloroplasty, Po...",General,Y,O


In [43]:
test_2['DX1']=test_2['Diagnosis Codes'].apply(lambda x: re.findall('\[(.*?)\]',x))
# print(data['DX'])
test_2['Procedure Codes1']=test_2['Case Procedures'].apply(lambda x: re.findall('\[(.*?)\]',x))
#print(data['Procedure Codes'])

test_2['DX_Codes'] = test_2['DX1'].str.get(0)
test_2['Procedure_Codes'] =  test_2['Procedure Codes1'].str.join(', ')
test_2.head()

,Diagnosis Codes,Case Procedures,Service,Case Indx,Sub Categ,DX1,Procedure Codes1,DX_Codes,Procedure_Codes
0,Pancreatic mass [K86.89],Laparoscopic Distal Pancreatectomy And Splenec...,General,Y,P-Distal,[K86.89],"[1078500003, 1075611941]",K86.89,"1078500003, 1075611941"
1,Ductal carcinoma in situ of right breast [D05.11],Right Lumpectomy [1075617330],General,Y,B-Partial M,[D05.11],[1075617330],D05.11,1075617330
2,Other chronic pancreatitis (HCC) [K86.1],"Total Pancreatectomy, Auto Islet Cell Transpla...",Transplant,Y,P-Total,[K86.1],"[1075612630, 1075611937]",K86.1,"1075612630, 1075611937"
3,Malignant neoplasm of lower-outer quadrant of ...,"Bilateral Total Mastectomy [1075617332], Right...",General,Y,B-M,"[C50.511, C50.919, Z90.13]","[1075617332, 1075611957, 1075617827, 107560976...",C50.511,"1075617332, 1075611957, 1075617827, 1075609766..."
4,Malignant neoplasm of lower third of esophagus...,"Laparoscopic Cholecystectomy, Pyloroplasty, Po...",General,Y,O,[C15.5],"[1075612301, 1075612588, 1075617799, 107561229...",C15.5,"1075612301, 1075612588, 1075617799, 1075612297..."


In [44]:
test_2 = test_2.drop(['Diagnosis Codes', 'Case Procedures', 'DX1', 'Procedure Codes1'], axis=1)
test_2.head()

,Service,Case Indx,Sub Categ,DX_Codes,Procedure_Codes
0,General,Y,P-Distal,K86.89,"1078500003, 1075611941"
1,General,Y,B-Partial M,D05.11,1075617330
2,Transplant,Y,P-Total,K86.1,"1075612630, 1075611937"
3,General,Y,B-M,C50.511,"1075617332, 1075611957, 1075617827, 1075609766..."
4,General,Y,O,C15.5,"1075612301, 1075612588, 1075617799, 1075612297..."


In [45]:
test_2_encoded = ignore_encoder.transform(test_2)
test_2_encoded.shape

(9, 247)

In [46]:
test_2_prediction = ovr_clf.predict(test_2_encoded)
pd.Series(test_2_prediction)

0     Pancreas
1     Pancreas
2     Pancreas
3     Pancreas
4    Esophagus
5         Lung
6     Pancreas
7     Pancreas
8     Pancreas
dtype: object

# Add Polynomial Features and Scale the data

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", OneVsRestClassifier(SVC(kernel="poly", degree=2, coef0=1, C=5)))
])

In [13]:
# Create new X and y since StandardScaler cannot scale sparse matrix. 
# We will set sparse in the OneHotEncoder to False.classification_report

non_sprase_encoder = OneHotEncoder(sparse=False)

X_non_sparse = non_sprase_encoder.fit_transform(X)
y_poly = label_binarize(y, classes=['Esophagus', 'Pancreas', 'Gastric', 'Other', 'Lung', 'O'])

print(X_non_sparse.shape)
print(y_poly.shape)

(305, 156)
(305, 6)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_non_sparse, y_poly, test_size=0.2, random_state=42)

In [15]:
poly_kernel_svm_clf.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf',
                 OneVsRestClassifier(estimator=SVC(C=5, coef0=1, degree=2,
                                                   kernel='poly')))])

In [16]:
print(classification_report(y_test, poly_kernel_svm_clf.predict(X_test), target_names=['Esophagus', 'Pancreas', 'Gastric', 'Other', 'Lung', 'O']))

              precision    recall  f1-score   support

   Esophagus       1.00      1.00      1.00        23
    Pancreas       1.00      1.00      1.00         7
     Gastric       1.00      1.00      1.00        11
       Other       0.00      0.00      0.00         0
        Lung       1.00      1.00      1.00        20
           O       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00        61
   macro avg       0.67      0.67      0.67        61
weighted avg       1.00      1.00      1.00        61
 samples avg       1.00      1.00      1.00        61



In [23]:
print(X_test.shape)
print(X_train.shape)

(61, 156)
(244, 156)


In [25]:
print(X_test[1])

[0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# LinearSVC model 

In [36]:
# Encode X and keep y as a 1d array. So we are not seperating the classes to be seperate features. LinearSVC doesn't train on y as a 2d array. 

X_encoded = non_sprase_encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(244, 156)
(61, 156)
(244,)
(61,)


In [39]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import PolynomialFeatures

lin_svc_clf = Pipeline([
    ("poly_feat", PolynomialFeatures(degree=2)),
    ("scaler", StandardScaler()),
    ("svc_clf", LinearSVC(C=10, dual=False, random_state=42))
])

In [40]:
lin_svc_clf.fit(X_train, y_train)

Pipeline(steps=[('poly_feat', PolynomialFeatures()),
                ('scaler', StandardScaler()),
                ('svc_clf', LinearSVC(C=10, dual=False, random_state=42))])

In [41]:
print(classification_report(y_test, lin_svc_clf.predict(X_test)))

              precision    recall  f1-score   support

   Esophagus       1.00      1.00      1.00        23
     Gastric       1.00      1.00      1.00        11
        Lung       1.00      1.00      1.00        20
    Pancreas       1.00      1.00      1.00         7

    accuracy                           1.00        61
   macro avg       1.00      1.00      1.00        61
weighted avg       1.00      1.00      1.00        61



In [44]:
y_train.unique()

array(['Lung', 'Pancreas', 'Gastric', 'Esophagus', 'Other', 'O'],
      dtype=object)

In [48]:
lin_svc_clf.predict(X_test[[1]])

array(['Lung'], dtype=object)

In [54]:
lin_svc_scores = lin_svc_clf.decision_function(X_test)
lin_svc_scores

array([[-0.52562228, -0.86066678,  0.75463809, -0.99196328, -0.98328239,
        -0.85465087],
       [-1.00000617, -0.99995366,  1.60245119, -0.99981807, -0.99982376,
        -0.99995392],
       [-0.83359237,  0.96737335, -0.93043959, -0.99438871, -0.99926791,
        -1.0048467 ],
       [-0.96972568, -0.9897694 ,  1.48794843, -0.99921544, -0.99573459,
        -0.98836163],
       [ 1.65286434, -1.00000244, -1.00037809, -0.99978695, -0.99980042,
        -0.99993398],
       [-1.27396666,  1.65883352, -0.94223145, -0.99254188, -1.01225041,
        -0.94519432],
       [-0.96624416, -0.98928925,  1.47373614, -0.99922627, -0.99881626,
        -0.99059228],
       [-1.00174485, -0.99995184,  1.60468533, -0.99981743, -0.99982249,
        -0.99998171],
       [ 1.62341267, -0.99989467, -1.00044633, -0.99978547, -0.99977683,
        -0.99992137],
       [-1.27396666,  1.65883352, -0.94223145, -0.99254188, -1.01225041,
        -0.94519432],
       [ 1.19039237, -0.9953587 , -0.91805541, -1.

# Run the Decsion_function on the SVC model. See what the scores are for each class. 

We also need new data the model hasn't seen to see if it is predicting/estimating correctly. 

In [56]:
lin_svc_scored = lin_svc_clf.score(X_test, y_test) 
lin_svc_scored

1.0